### lncRNAs quantified? realive to background
1. construct backgorund gtf
2. map reads with rnaseqc
3. compare to see if lncRNAs are quantified

In [2]:
import pandas as pd
import seaborn as sns
import pyranges as pr
import numpy as np

#### create background gtf

In [3]:
gencode_v47_path = "/home/klawren/oak/gtex/data/references/gencode.v47.annotation.gtf"
gencode_v47=pr.read_gtf(gencode_v47_path).as_df()

In [4]:
gencode_v47

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_type,...,exon_number,exon_id,transcript_support_level,havana_transcript,hgnc_id,havana_gene,ont,protein_id,ccdsid,artif_dupl
0,chr1,HAVANA,gene,11120,24894,.,+,.,ENSG00000290825.2,lncRNA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chr1,HAVANA,transcript,11120,14413,.,+,.,ENSG00000290825.2,lncRNA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chr1,HAVANA,exon,11120,11211,.,+,.,ENSG00000290825.2,lncRNA,...,1,ENSE00004248723.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,chr1,HAVANA,exon,12009,12227,.,+,.,ENSG00000290825.2,lncRNA,...,2,ENSE00004248735.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,chr1,HAVANA,exon,12612,12721,.,+,.,ENSG00000290825.2,lncRNA,...,3,ENSE00003582793.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4105480,chrY,HAVANA,exon,57214349,57214397,.,-,.,ENSG00000292371.1,unprocessed_pseudogene,...,1,ENSE00004015123.1,NA,OTTHUMT00000058841.1,HGNC:37115,OTTHUMG00000022678.1,PGO:0000005,NaN,NaN,NaN
4105481,chrY,HAVANA,exon,57213879,57213964,.,-,.,ENSG00000292371.1,unprocessed_pseudogene,...,2,ENSE00004015124.1,NA,OTTHUMT00000058841.1,HGNC:37115,OTTHUMG00000022678.1,PGO:0000005,NaN,NaN,NaN
4105482,chrY,HAVANA,exon,57213525,57213602,.,-,.,ENSG00000292371.1,unprocessed_pseudogene,...,3,ENSE00004015125.1,NA,OTTHUMT00000058841.1,HGNC:37115,OTTHUMG00000022678.1,PGO:0000005,NaN,NaN,NaN
4105483,chrY,HAVANA,exon,57213203,57213357,.,-,.,ENSG00000292371.1,unprocessed_pseudogene,...,4,ENSE00004015126.1,NA,OTTHUMT00000058841.1,HGNC:37115,OTTHUMG00000022678.1,PGO:0000005,NaN,NaN,NaN


"In order to use XBSeq for testing DE, we need to run HTSeq twice to measure the reads mapped to exonic regions (observed signal) and non-exonic regions (background noise). Firstly, we need to construct the gtf annotation file to measure the background noise:

     \item {Download refFlat table from UCSC database (http://genome.ucsc.edu) and create the preliminary list of gene-free regions,}
	 \item {Download tables of (a) all_mrna; (b) ensGene; (c) pseudoYale60Gene; (d) vegaGene;, (e)xenoMrna, and (f) xenoRefGene from UCSC database and remove 
	 regions appear in any of them from the gene-free regions,}
     \item {To guarantee gene-free regions are far enough from exonic regions, trim 100 bps from both sides of intronic regions and 1,000 bps from both sides 
	 of inter-genic regions,}
     \item {Shift each exon of a gene to the right nearest gene-free region.  Most of the shifted genes remain the same as the original structures of the genes,}
     \item {If the nearby gene-free region is too short, we may only preserve the exon size features but not the whole gene structure. The priority of shifting 
	 a region is: i) nearest right gene-free region, 2) nearest left gene-free region; 3) the second right nearest gene-free region and so on until the shift 
	 region of the original exon fits, and}
     \item {Shift each exon of a gene to the right nearest gene-free region.  Most of the shifted genes remain the same as the original structures of the genes,}
     \item {At last, we considered the shifted regions as the non-exonic regions for each gene and a final .gtf file was created}
